In [ ]:
function VFI_new(ae; hide=false)
    @unpack toldiffvalue = ae
    @unpack q, Vd, Vc, V_bar, V_bar_d, Vd0, d = ae #unpack qa=1/rf
    
    Vc_upd = similar(Vc)
    Vd_upd = similar(Vd)
    valuediff=1.
    iter=1
    
    while valuediff>toldiffvalue
        copyto!(Vc_upd, Vc)
        copyto!(Vd_upd, Vd)
        
        # 1. V_bar,Vd0,d <- Update_Vbar_d(Vc, Vd)
        update_Vbar_d!(ae, Vc, Vd, V_bar, V_bar_d, Vd0, d)
        
        # 2. Vc, Vd  <- Bellman_VcVd!(V_bar,Vd0,q)
        Bellman_VcVd!(ae, Vc, Vd, V_bar, V_bar_d, Vd0, q)
        
        # 3. q <- update_prices!(q,index_b,index_bs)
        update_prices!(ae, q, d)
        
        Vc_dif = abs.(Vc - Vc_upd)
        Vd_dif = abs.(Vd - Vd_upd)        
        valuediff = max(maximum(Vc_dif), maximum(Vd_dif))
        iter += 1
        
        if hide == false
            if valuediff<=toldiffvalue
                println("Convergence OK, (iteration, valuediff):", (iter, round(valuediff, digits=7)))
                break
            end
        else
            if valuediff<=toldiffvalue
                # println("Convergence OK, (iteration, valuediff):", (iter, round(valuediff, digits=7)))
                break
            end            
        end
            
        
        #if iter % 15 == 0
        #    println("Iter, valuediff:", (iter, valuediff))
        #end 
        
        if iter>200
            println("Iter>200 -> Break. valuediff=", (round(valuediff, digits=7), toldiffvalue))
            break
        end
        
    end
end

In [ ]:
function update_Vbar_d!(ae, Vc, Vd, V_bar, V_bar_d, Vd0, d) # El output es V_bar y d. Input: i_bp y Vd0
    # In: Vc,Vd,Vd0 |i_bp -> Out: Vbar,Vd0,d
    
    @unpack Ny, Nb, Nbs, grid_b, Nc = ae
    indexnodebt = searchsortedfirst(grid_b, 0.)
    
    for i_bs = 1:Nbs
        for i_y = 1:Ny
            for i_b = 1:Nb
                for i_c = 1:Nc
                    Vd0[i_y, i_bs, i_c]        = Vd[indexnodebt, i_y, i_bs, i_c]
                    V_bar[i_b, i_y, i_bs, i_c] = max(Vc[i_b, i_y, i_bs, i_c], Vd0[i_y, i_bs, i_c])
                    V_bar_d[i_y, i_bs, i_c]    = V_bar[indexnodebt, i_y, i_bs, i_c]
                    #update default set
                    if Vc[i_b, i_y, i_bs, i_c] > Vd[indexnodebt, i_y, i_bs, i_c]
                        d[i_b, i_y, i_bs, i_c] = 0 
                    else
                        d[i_b, i_y, i_bs, i_c] = 1 
                    end    
                end
            end
        end
    end
end

In [ ]:
function Bellman_Update_bystate!(ae, V_bar, V_bar_d, Vd0, q, states)
    
    @unpack Λ, Γ, τ, z, Δ, ψ, ξ, qs, σ, π, μ, β, Ny, Nc = ae
    @unpack Nb, Nbs, grid_yd, grid_yc, grid_b, grid_bs, grid_c  = ae
    @unpack restriccion, y_mean, x_mean, Θ, b_limit, Bs_mean = ae #Restricción
    i_y, i_b, i_bs, i_c = states
    
    #Escenario economía
    #Escenario malo
    if (grid_yc[i_y] < 1) && (grid_c[i_c] < 1) 
        escenario = 0
    #Escenario bueno
    elseif (grid_yc[i_y] > 1) && (grid_c[i_c] > 1) 
        escenario = 1
    #Escenarios intermedios
    else  
        escenario = 2
    end          
    
    

    l_c = (grid_yc[i_y]/((1+τ)) )^(1/ψ) 
    l_d = (grid_yd[i_y]/((1+τ)) )^(1/ψ) 
    
    y_c = grid_yc[i_y] * l_c
    y_d = grid_yd[i_y] * l_d

    c_c = grid_yc[i_y] * l_c/(1+τ) 
    c_d = grid_yd[i_y] * l_d/(1+τ)                   
    
    y_d_full = y_d + z*grid_c[i_c]
    y_c_full = y_c + z*grid_c[i_c] 
    
    for i_bsp=1:Nbs #loop begins here ends after vc
        g_d = y_d*τ + z*grid_c[i_c] + grid_bs[i_bs] - qs*grid_bs[i_bsp]

        """ Caso 0: modelo óptimo """
        if restriccion == 0 
        
            # """Value function in default  """ 
            if (g_d>0) & ((c_d-((l_d^(1+ψ))/(1+ψ)))>0) 
                u_d = π*(g_d^(1-σ))/(1-σ) + (1-π)*(((c_d-((l_d^(1+ψ))/(1+ψ)))^(1-σ))/(1-σ))
                Evd = 0.0
                for i = 1:Ny
                    for j = 1:Nc
                        Evd = Evd + μ*(Λ[i_y, i]*Γ[i_c, j]*V_bar_d[i, i_bsp, j]) + (1-μ)*(Λ[i_y, i]*Γ[i_c, j]*Vd0[i, i_bsp, j])
                    end
                end            
                ae.vd[i_bsp]= u_d + β*Evd 
            else 
                ae.vd[i_bsp]= -666
            end 

            #"""Value function credit  """ 
            for i_bp = 1:Nb # every point for grid_b in the next period
                g_c = y_c*τ + z*grid_c[i_c] + grid_bs[i_bs] - qs*grid_bs[i_bsp] + Δ*grid_b[i_b] -
                        q[i_bp, i_y, i_bsp, i_c]*(grid_b[i_bp] - (1-Δ)*grid_b[i_b])

                #Update vc
                if (g_c>0) & ((c_c-((l_c^(1+ψ))/(1+ψ)))>0)
                    u_c = π*(g_c^(1-σ))/(1-σ) + (1-π)*(((c_c-((l_c^(1+ψ))/(1+ψ)))^(1-σ))/(1-σ))
                    Evc = 0.0
                    for i = 1:Ny
                        for j = 1:Nc
                            Evc = Evc + Λ[i_y, i] * Γ[i_c, j] * V_bar[i_bp, i, i_bsp, j]
                        end
                    end                   
                    ae.vc[i_bp, i_bsp] = u_c + β*Evc
                else 
                    ae.vc[i_bp, i_bsp] = -666
                end  
            end
            
        """ --------- Modelos con reglas sin cláusulas de escape ----------- """ 
        
        """ Caso 1: modelo con regla de deuda """
        elseif restriccion == 1
            # """Value function in default  """ 
            if (g_d>0) & ((c_d-((l_d^(1+ψ))/(1+ψ)))>0)  
                u_d = π*(g_d^(1-σ))/(1-σ) + (1-π)*(((c_d-((l_d^(1+ψ))/(1+ψ)))^(1-σ))/(1-σ))
                Evd = 0.0
                for i = 1:Ny
                    for j = 1:Nc
                        Evd = Evd + μ*(Λ[i_y, i]*Γ[i_c, j]*V_bar_d[i, i_bsp, j]) + (1-μ)*(Λ[i_y, i]*Γ[i_c, j]*Vd0[i, i_bsp, j])
                    end
                end            
                ae.vd[i_bsp]= u_d + β*Evd 
            else 
                ae.vd[i_bsp]= -666
            end 

            #"""Value function credit  """ 
            for i_bp = 1:Nb # every point for grid_b in the next period
                g_c = y_c*τ + z*grid_c[i_c] + grid_bs[i_bs] - qs*grid_bs[i_bsp] + Δ*grid_b[i_b] -
                        q[i_bp, i_y, i_bsp, i_c]*(grid_b[i_bp] - (1-Δ)*grid_b[i_b])

                #Update vc
                if (grid_b[i_bp]/y_c_full>b_limit) & (g_c>0) & ((c_c-((l_c^(1+ψ))/(1+ψ)))>0)
                    u_c = π*(g_c^(1-σ))/(1-σ) + (1-π)*(((c_c-((l_c^(1+ψ))/(1+ψ)))^(1-σ))/(1-σ))
                    Evc = 0.0
                    for i = 1:Ny
                        for j = 1:Nc
                            Evc = Evc + Λ[i_y, i] * Γ[i_c, j] * V_bar[i_bp, i, i_bsp, j]
                        end
                    end                   
                    ae.vc[i_bp, i_bsp] = u_c + β*Evc
                else 
                    ae.vc[i_bp, i_bsp] = -666
                end  
            end
                    
            
        """ --------- Modelos con reglas de escape ----------- """  
                        
        """ Caso 2: modelo con regla deuda y cláusulas de escape"""               
        elseif restriccion == 2
            
            if escenario == 0 #Escenario malo
                
                # """Value function in default  """ 
                if (g_d>0) & (g_d<=τ*y_d + z*grid_c[i_c] + grid_bs[i_bs] - qs*grid_bs[i_bsp])  & ((c_d-((l_d^(1+ψ))/(1+ψ)))>0) 
                    u_d = π*(g_d^(1-σ))/(1-σ) + (1-π)*(((c_d-((l_d^(1+ψ))/(1+ψ)))^(1-σ))/(1-σ))
                    Evd = 0.0
                    for i = 1:Ny
                        for j = 1:Nc
                            Evd = Evd + μ*(Λ[i_y, i]*Γ[i_c, j]*V_bar_d[i, i_bsp, j]) + (1-μ)*(Λ[i_y, i]*Γ[i_c, j]*Vd0[i, i_bsp, j])
                        end
                    end            
                    ae.vd[i_bsp]= u_d + β*Evd 
                else 
                    ae.vd[i_bsp]= -666
                end 

                #"""Value function credit  """ 
                for i_bp = 1:Nb # every point for grid_b in the next period
                    g_c = y_c*τ + z*grid_c[i_c] + grid_bs[i_bs] - qs*grid_bs[i_bsp] + Δ*grid_b[i_b] -
                            q[i_bp, i_y, i_bsp, i_c]*(grid_b[i_bp] - (1-Δ)*grid_b[i_b])

                    #Update vc
                    if (g_c>0) & ((c_c-((l_c^(1+ψ))/(1+ψ)))>0)
                        u_c = π*(g_c^(1-σ))/(1-σ) + (1-π)*(((c_c-((l_c^(1+ψ))/(1+ψ)))^(1-σ))/(1-σ))
                        Evc = 0.0
                        for i = 1:Ny
                            for j = 1:Nc
                                Evc = Evc + Λ[i_y, i] * Γ[i_c, j] * V_bar[i_bp, i, i_bsp, j]
                            end
                        end                   
                        ae.vc[i_bp, i_bsp] = u_c + β*Evc
                    else 
                        ae.vc[i_bp, i_bsp] = -666
                    end  
                end                    
                
                
            else ##Escenario intermedio y bueno
                
                # """Value function in default  """ 
                # if (g_d>0) & (g_d<=τ*y_d + z*grid_c[i_c] + grid_bs[i_bs] - qs*grid_bs[i_bsp]) & (g_d <= τ*y_mean + x_mean + (1-qs)*grid_bs[i_bs] - Θ*y_d_full) & ((c_d-((l_d^(1+ψ))/(1+ψ)))>0) 
                if (g_d>0) & (g_d<=τ*y_d + z*grid_c[i_c] + grid_bs[i_bs] - qs*grid_bs[i_bsp]) & ((c_d-((l_d^(1+ψ))/(1+ψ)))>0) 
                    u_d = π*(g_d^(1-σ))/(1-σ) + (1-π)*(((c_d-((l_d^(1+ψ))/(1+ψ)))^(1-σ))/(1-σ))
                    Evd = 0.0
                    for i = 1:Ny
                        for j = 1:Nc
                            Evd = Evd + μ*(Λ[i_y, i]*Γ[i_c, j]*V_bar_d[i, i_bsp, j]) + (1-μ)*(Λ[i_y, i]*Γ[i_c, j]*Vd0[i, i_bsp, j])
                        end
                    end            
                    ae.vd[i_bsp]= u_d + β*Evd 
                else 
                    ae.vd[i_bsp]= -666
                end 

                #"""Value function credit  """ 
                for i_bp = 1:Nb # every point for grid_b in the next period
                    g_c = y_c*τ + z*grid_c[i_c] + grid_bs[i_bs] - qs*grid_bs[i_bsp] + Δ*grid_b[i_b] -
                            q[i_bp, i_y, i_bsp, i_c]*(grid_b[i_bp] - (1-Δ)*grid_b[i_b])

                    #Update vc
                    # if (grid_b[i_bp]/y_c_full>b_limit) & (g_c>0) & (g_c <= τ*y_mean + x_mean + (1-q[i_bp, i_y, i_bsp, i_c])*grid_b[i_bp] + (1-qs)*grid_bs[i_bs] - Θ*y_c_full) & ((c_c-((l_c^(1+ψ))/(1+ψ)))>0)
                    if (grid_b[i_bp]/y_c_full>b_limit) & (g_c>0) & ((c_c-((l_c^(1+ψ))/(1+ψ)))>0)
                        u_c = π*(g_c^(1-σ))/(1-σ) + (1-π)*(((c_c-((l_c^(1+ψ))/(1+ψ)))^(1-σ))/(1-σ))
                        Evc = 0.0
                        for i = 1:Ny
                            for j = 1:Nc
                                Evc = Evc + Λ[i_y, i] * Γ[i_c, j] * V_bar[i_bp, i, i_bsp, j]
                            end
                        end                   
                        ae.vc[i_bp, i_bsp] = u_c + β*Evc
                    else 
                        ae.vc[i_bp, i_bsp] = -666
                    end  
                end   
            end
                        
        """ Caso 3: modelo con regla deuda y cláusulas de escape en escenario adverso e intermedio"""               
        elseif restriccion == 3
            
            if (escenario == 0) || (escenario == 2) #Escenario malo o intermedio
                
                # """Value function in default  """ 
                if (g_d>0) & (g_d<=τ*y_d + z*grid_c[i_c] + grid_bs[i_bs] - qs*grid_bs[i_bsp])  & ((c_d-((l_d^(1+ψ))/(1+ψ)))>0) 
                    u_d = π*(g_d^(1-σ))/(1-σ) + (1-π)*(((c_d-((l_d^(1+ψ))/(1+ψ)))^(1-σ))/(1-σ))
                    Evd = 0.0
                    for i = 1:Ny
                        for j = 1:Nc
                            Evd = Evd + μ*(Λ[i_y, i]*Γ[i_c, j]*V_bar_d[i, i_bsp, j]) + (1-μ)*(Λ[i_y, i]*Γ[i_c, j]*Vd0[i, i_bsp, j])
                        end
                    end            
                    ae.vd[i_bsp]= u_d + β*Evd 
                else 
                    ae.vd[i_bsp]= -666
                end 

                #"""Value function credit  """ 
                for i_bp = 1:Nb # every point for grid_b in the next period
                    g_c = y_c*τ + z*grid_c[i_c] + grid_bs[i_bs] - qs*grid_bs[i_bsp] + Δ*grid_b[i_b] -
                            q[i_bp, i_y, i_bsp, i_c]*(grid_b[i_bp] - (1-Δ)*grid_b[i_b])

                    #Update vc
                    if (g_c>0) & ((c_c-((l_c^(1+ψ))/(1+ψ)))>0)
                        u_c = π*(g_c^(1-σ))/(1-σ) + (1-π)*(((c_c-((l_c^(1+ψ))/(1+ψ)))^(1-σ))/(1-σ))
                        Evc = 0.0
                        for i = 1:Ny
                            for j = 1:Nc
                                Evc = Evc + Λ[i_y, i] * Γ[i_c, j] * V_bar[i_bp, i, i_bsp, j]
                            end
                        end                   
                        ae.vc[i_bp, i_bsp] = u_c + β*Evc
                    else 
                        ae.vc[i_bp, i_bsp] = -666
                    end  
                end                    
                
                
            else ##Escenario bueno
                
                # """Value function in default  """ 
                # if (g_d>0) & (g_d<=τ*y_d + z*grid_c[i_c] + grid_bs[i_bs] - qs*grid_bs[i_bsp]) & (g_d <= τ*y_mean + x_mean + (1-qs)*grid_bs[i_bs] - Θ*y_d_full) & ((c_d-((l_d^(1+ψ))/(1+ψ)))>0) 
                if (g_d>0) & (g_d<=τ*y_d + z*grid_c[i_c] + grid_bs[i_bs] - qs*grid_bs[i_bsp]) & ((c_d-((l_d^(1+ψ))/(1+ψ)))>0) 
                    u_d = π*(g_d^(1-σ))/(1-σ) + (1-π)*(((c_d-((l_d^(1+ψ))/(1+ψ)))^(1-σ))/(1-σ))
                    Evd = 0.0
                    for i = 1:Ny
                        for j = 1:Nc
                            Evd = Evd + μ*(Λ[i_y, i]*Γ[i_c, j]*V_bar_d[i, i_bsp, j]) + (1-μ)*(Λ[i_y, i]*Γ[i_c, j]*Vd0[i, i_bsp, j])
                        end
                    end            
                    ae.vd[i_bsp]= u_d + β*Evd 
                else 
                    ae.vd[i_bsp]= -666
                end 

                #"""Value function credit  """ 
                for i_bp = 1:Nb # every point for grid_b in the next period
                    g_c = y_c*τ + z*grid_c[i_c] + grid_bs[i_bs] - qs*grid_bs[i_bsp] + Δ*grid_b[i_b] -
                            q[i_bp, i_y, i_bsp, i_c]*(grid_b[i_bp] - (1-Δ)*grid_b[i_b])

                    #Update vc
                    # if (grid_b[i_bp]/y_c_full>b_limit) & (g_c>0) & (g_c <= τ*y_mean + x_mean + (1-q[i_bp, i_y, i_bsp, i_c])*grid_b[i_bp] + (1-qs)*grid_bs[i_bs] - Θ*y_c_full) & ((c_c-((l_c^(1+ψ))/(1+ψ)))>0)
                    if (grid_b[i_bp]/y_c_full>b_limit) & (g_c>0) & ((c_c-((l_c^(1+ψ))/(1+ψ)))>0)
                        u_c = π*(g_c^(1-σ))/(1-σ) + (1-π)*(((c_c-((l_c^(1+ψ))/(1+ψ)))^(1-σ))/(1-σ))
                        Evc = 0.0
                        for i = 1:Ny
                            for j = 1:Nc
                                Evc = Evc + Λ[i_y, i] * Γ[i_c, j] * V_bar[i_bp, i, i_bsp, j]
                            end
                        end                   
                        ae.vc[i_bp, i_bsp] = u_c + β*Evc
                    else 
                        ae.vc[i_bp, i_bsp] = -666
                    end  
                end   
            end
            
        end  #restriccion
    end # loops end here when bs 

    # Maximization 
    Vc_max, index_c_max = findmax(ae.vc)
    Vd_max, index_d_max = findmax(ae.vd) 
    
  
    return Vc_max, Vd_max, index_c_max, index_d_max
    
end


function Bellman_VcVd!(ae, Vc, Vd, V_bar, V_bar_d, Vd0, q)
    @unpack Nb, Nbs, Ny, Nc = ae
    
    for i_bs = 1:Nbs  
        for i_y=1:Ny 
            for i_b = 1:Nb
                for i_c = 1:Nc
                    #  Vd,Vc,d <- Bellman(Vd,Vc,d,q)
                    states = (i_y, i_b, i_bs, i_c)
                    Vc_max, Vd_max, index_c_max, index_d_max = Bellman_Update_bystate!(ae, V_bar, V_bar_d, Vd0, q, states)

                    Vc[i_b, i_y, i_bs, i_c] = Vc_max 
                    Vd[i_b, i_y, i_bs, i_c] = Vd_max 

                    # Create indexes for b and bs
                    ae.index_b_c[i_b, i_y, i_bs, i_c]  = index_c_max[1]
                    ae.index_bs_c[i_b, i_y, i_bs, i_c] = index_c_max[2]
                    ae.index_bs_d[i_b, i_y, i_bs, i_c] = index_d_max[1]
                end
            end
        end
    end
end

In [ ]:
function update_prices!(ae, q, d)
    @unpack Nb, Nbs, Ny, Nc, M, Λ, Γ, ξ, rf, Δ, prob_def = ae
       
    for i_y=1:Ny         #a
        for i_c = 1:Nc   #x
            for i_bp = 1:Nb
                for i_bsp = 1:Nbs #short asset grid
                    q[i_bp, i_y, i_bsp, i_c] = 0
                    prob_def[i_bp, i_y, i_bsp, i_c] = 0.
                    z = 1 
                    while z <= Ny  # future output : y'
                        x = 1
                        while x <= Nc # future copper 
                            i_bpp  = ae.index_b_c[i_bp, z, i_bsp, x]
                            i_bspp = ae.index_bs_c[i_bp, z, i_bsp, x]
                            prob_def[i_bp, i_y, i_bsp, i_c] =  prob_def[i_bp, i_y, i_bsp, i_c] + Λ[i_y, z]*Γ[i_c, x]*d[i_bp, z, i_bsp, x]                     

                            # q[i_bp, i_y, i_bsp, i_c] = q[i_bp, i_y, i_bsp, i_c] + M[z, i_y]*Λ[i_y, z]*Γ[i_c, x]*(1-d[i_bp, z, i_bsp, x])*
                            #                            (Δ + (1-Δ)*q[i_bpp, z, i_bspp, x])/(1+rf)   
                            q[i_bp, i_y, i_bsp, i_c] = q[i_bp, i_y, i_bsp, i_c] + M[z, i_y]*Λ[i_y, z]*Γ[i_c, x]*(1-d[i_bp, z, i_bsp, x])*
                                                       (Δ + (1-Δ)*q[i_bpp, z, i_bspp, x])/(1+rf)                               
                            x = x + 1 
                        end
                        z = z + 1     
                    end 
                end 
            end 
        end
    end   
end

## Policy Function

In [ ]:
function policy!(ae)
    @unpack ψ, q, Nb, Nbs, Ny, Nc, ξ, grid_b, grid_bs = ae
    @unpack index_b_c, index_b_d, index_bs_c, index_bs_d, τ, z, Δ, qs = ae
    @unpack b_c, b_d, bs_c, bs_d, g_c, g_d, c_c, c_d = ae
    @unpack grid_yc, grid_yd, grid_c, l_c, l_d, y_d_full, y_c_full = ae

    g_c2 = zeros(Nb, Nbs)
    g_d2 = zeros(Nbs)
    
    for i_y = 1:Ny  
        l_c[i_y] = (grid_yc[i_y]/((1+τ)) )^(1/ψ) 
        l_d[i_y] = (grid_yd[i_y]/((1+τ)) )^(1/ψ) 

        y_c = grid_yc[i_y] * l_c[i_y]
        y_d = grid_yd[i_y] * l_d[i_y]

        c_c[i_y] = grid_yc[i_y] * l_c[i_y]/(1+τ) 
        c_d[i_y] = grid_yd[i_y] * l_d[i_y]/(1+τ)     
       
        for i_c = 1:Nc
            # y_d_full = y_d + z*grid_c[i_c]
            # y_c_full = y_c + z*grid_c[i_c]   
            
            y_d_full[i_y, i_c] = y_d + z*grid_c[i_c]
            y_c_full[i_y, i_c] = y_c + z*grid_c[i_c]               
            
            
            for i_bs = 1:Nbs 
                for i_bsp = 1:Nbs         
                    g_d2[i_bsp] = y_d*τ + z*grid_c[i_c] + grid_bs[i_bs] - qs*grid_bs[i_bsp]
                end

                for i_b = 1:Nb 
                    #grid_b and taxes
                    b_c[i_b, i_y, i_bs, i_c]  = grid_b[Int(index_b_c[i_b, i_y, i_bs, i_c])] 
                    bs_c[i_b, i_y, i_bs, i_c] = grid_bs[Int(index_bs_c[i_b, i_y, i_bs, i_c])] 
                    bs_d[i_b, i_y, i_bs, i_c] = grid_bs[Int(index_bs_d[i_b, i_y, i_bs, i_c])]  

                    #Policy
                    for i_bsp = 1:Nbs
                        for i_bp = 1:Nb # every point for grid_b in the next period
                            g_c2[i_bp, i_bsp] = y_c*τ + z*grid_c[i_c] + grid_bs[i_bs] - 
                                                qs*grid_bs[i_bsp] + Δ*grid_b[i_b] -
                                                q[i_bp, i_y, i_bsp, i_c]*(grid_b[i_bp] - (1-Δ)*grid_b[i_b])                        
                        end
                    end
                    g_d[i_y, i_bs, i_c] = g_d2[index_bs_d[i_b, i_y, i_bs, i_c]]
                    g_c[i_b, i_y, i_bs, i_c] = g_c2[index_b_c[i_b, i_y, i_bs, i_c], index_bs_c[i_b, i_y, i_bs, i_c]]
                end
            end  
        end
    end   
end